In [1]:
import sys
sys.path.append("../../Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages")

In [3]:
class Dataset:
    def __init__(self, data:dict,mutations=[]):
        self._data=data
        self._original=data
        self._mutations=mutations
    def reset_data(self):
        self.data=self.original
        self.mutations=[]
    def addMutation(self,mutation_list:list):
        self._mutations.append(mutation_list)
    def keys(self):
        return self._data.keys()
    def values(self):
        return self._data.values()
    def items(self):
        return self._data.items()